In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import nltk.data
from nltk.tokenize import BlanklineTokenizer
import sys
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
# test = open("/content/drive/MyDrive/Hybridization/input/en-hi-wsd-3L/output_en-hi_WSD-500_intrain.txt", "r")
# reference = open("/content/drive/MyDrive/Hybridization/output/en-hi/ref_en-hi-500-intrain.txt", "r")
# reference = open("/content/drive/MyDrive/MTE_Files/METEOR/parallel-n/english.txt","r")
# test = open("/content/drive/MyDrive/MTE_Files/METEOR/parallel-n/hindi.txt","r")

# reference = open("/content/drive/MyDrive/MTdata/references(human).txt")
# test = open("/content/drive/MyDrive/MTdata/hypothesis(machine).txt")


test = "/content/drive/MyDrive/MTdata/Hindi/yandex_hypothesis.txt"
reference = "/content/drive/MyDrive/MTdata/Hindi/references(human).txt"

In [4]:
list1 = []
list2 = []
for line in reference:
    #print(line)
    list1.append(BlanklineTokenizer().tokenize(line))
for line in test:
    #print(line)
    list2.append(BlanklineTokenizer().tokenize(line))
# reference.close()
# test.close()

In [5]:
s1=[]
s2=[]
for ele in list1:
    for elem in ele:
        s1.append(" ".join([str(elem) for elem in ele]))
for ele in list2:
    for elem in ele:
        s2.append(" ".join([str(elem) for elem in ele]))

In [6]:
le=len(s2)
te = len(s1)
print(le)
print(te)

57
57


In [7]:
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
from itertools import chain, product


def _generate_enums(hypothesis, reference, preprocess=str.lower):
    hypothesis_list = list(enumerate(preprocess(hypothesis).split()))
    reference_list = list(enumerate(preprocess(reference).split()))
    return hypothesis_list, reference_list

def _enum_stem_match(enum_hypothesis_list, enum_reference_list, stemmer = PorterStemmer()):
    stemmed_enum_list1 = [(word_pair[0],stemmer.stem(word_pair[1])) \
                          for word_pair in enum_hypothesis_list]

    stemmed_enum_list2 = [(word_pair[0],stemmer.stem(word_pair[1])) \
                          for word_pair in enum_reference_list]

    word_match, enum_unmat_hypo_list, enum_unmat_ref_list = \
                    _match_enums(stemmed_enum_list1, stemmed_enum_list2)

    enum_unmat_hypo_list = list(zip(*enum_unmat_hypo_list)) if len(enum_unmat_hypo_list)>0 else []

    enum_unmat_ref_list = list(zip(*enum_unmat_ref_list)) if len(enum_unmat_ref_list)>0 else []

    enum_hypothesis_list = list(filter(lambda x:x[0] not in enum_unmat_hypo_list,  
                                       enum_hypothesis_list))

    enum_reference_list = list(filter(lambda x:x[0] not in enum_unmat_ref_list, 
                                      enum_reference_list))

    return word_match, enum_hypothesis_list, enum_reference_list




def _match_enums(enum_hypothesis_list, enum_reference_list):
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        for j in range(len(enum_reference_list))[::-1]:
            if enum_hypothesis_list[i][1] == enum_reference_list[j][1]:
                word_match.append((enum_hypothesis_list[i][0],enum_reference_list[j][0]))
                (enum_hypothesis_list.pop(i)[1],enum_reference_list.pop(j)[1])
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def _enum_wordnetsyn_match(enum_hypothesis_list, enum_reference_list, wordnet = wordnet):
    word_match = []
    for i in range(len(enum_hypothesis_list))[::-1]:
        hypothesis_syns = set(chain(*[[lemma.name() for lemma in synset.lemmas() 
                                        if lemma.name().find('_')<0] 
                                       for synset in \
                                           wordnet.synsets(
                                               enum_hypothesis_list[i][1])]
                                               )).union({enum_hypothesis_list[i][1]})
        for j in range(len(enum_reference_list))[::-1]:
            if enum_reference_list[j][1] in hypothesis_syns:
                word_match.append((enum_hypothesis_list[i][0],enum_reference_list[j][0]))
                enum_hypothesis_list.pop(i),enum_reference_list.pop(j)
                break
    return word_match, enum_hypothesis_list, enum_reference_list


def _count_chunks(matches):
    i=0
    chunks = 1
    while(i<len(matches)-1):
        if (matches[i+1][0]==matches[i][0]+1) and (matches[i+1][1]==matches[i][1]+1):
            i+=1
            continue
        i+=1
        chunks += 1
    return chunks






def _enum_allign_words(enum_hypothesis_list, enum_reference_list, 
                       stemmer=PorterStemmer(), wordnet = wordnet):
    exact_matches, enum_hypothesis_list, enum_reference_list = \
        _match_enums(enum_hypothesis_list, enum_reference_list)

    stem_matches, enum_hypothesis_list, enum_reference_list = \
        _enum_stem_match(enum_hypothesis_list, enum_reference_list,
                         stemmer = stemmer)

    wns_matches, enum_hypothesis_list, enum_reference_list = \
        _enum_wordnetsyn_match(enum_hypothesis_list, enum_reference_list, 
                               wordnet = wordnet)

    return (sorted(exact_matches + stem_matches + wns_matches, key=lambda wordpair:wordpair[0]),
            enum_hypothesis_list, enum_reference_list)




def single_meteor_score(reference, 
                        hypothesis, 
                        preprocess = str.lower, 
                        stemmer = PorterStemmer(), 
                        wordnet = wordnet, 
                        alpha=0.9, 
                        beta=3, 
                        gamma=0.5):
    enum_hypothesis, enum_reference = _generate_enums(hypothesis, 
                                                      reference, 
                                                      preprocess = preprocess)
    #print(enum_hypothesis, enum_reference)
    
    translation_length = len(enum_hypothesis)
    reference_length = len(enum_reference)
    matches, _, _ = _enum_allign_words(enum_hypothesis, enum_reference)
    
    #print(matches, _, _)
    matches_count = len(matches)
    try:
        precision = float(matches_count)/translation_length
        recall = float(matches_count)/reference_length
        fmean = (precision*recall)/(alpha*precision+(1-alpha)*recall)
        chunk_count = float(_count_chunks(matches))
        frag_frac = chunk_count/matches_count
    except ZeroDivisionError:
        return 0.0        
    penalty = gamma*frag_frac**beta
    return (1-penalty)*fmean


In [8]:
lop=0
scor=[]
coun=0
for lop in range(le):
    scor.append(single_meteor_score(s1[lop],s2[lop]))

In [9]:
f1 = open('/content/meteor_blank_score_ur1.txt', "w")
lope=0
su=0
for lope in range(le):
    print(f"{lope+1}.   {scor[lope]}")
    su=su+scor[lope]
    f1.write(str(scor[lope])+'\n') #Sentence_score
f1.close()
su=su/le
su #Corpus_score

1.   0.5
2.   0.5
3.   0.5
4.   0.5
5.   0.5
6.   0.5
7.   0.5
8.   0.5
9.   0.5
10.   0.5
11.   0.5
12.   0.5
13.   0.5
14.   0.5
15.   0.5
16.   0.5
17.   0.5
18.   0.5
19.   0.5
20.   0.5
21.   0.5
22.   0.5
23.   0.5
24.   0.5
25.   0.5
26.   0.5
27.   0.5
28.   0.5
29.   0.5
30.   0.5
31.   0.5
32.   0.5
33.   0.5
34.   0.5
35.   0.5
36.   0.5
37.   0.0
38.   0.0
39.   0.0
40.   0.0
41.   0.0
42.   0.0
43.   0.0
44.   0.0
45.   0.0
46.   0.0
47.   0.0
48.   0.0
49.   0.0
50.   0.0
51.   0.0
52.   0.0
53.   0.0
54.   0.5
55.   0.5
56.   0.5
57.   0.5


0.3508771929824561

In [10]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
f5 = open('/content/meteor_blank_score_ur1-yh.txt', "a+")
with open(test, "r") as f1, open(reference, "r") as f2:
  lines1 = [line.rstrip('\n') for line in f1]
  print(len(lines1))
  lines2 = [line.rstrip('\n') for line in f2]
  print(len(lines2))
  for l1,l2 in zip(lines1,lines2):
    tokens_ref=word_tokenize(l2)
    #print(tokens_ref)
    tokens_translate=word_tokenize(l1)
    #print(tokens_translate)
    score = nltk.translate.meteor_score.single_meteor_score(tokens_ref, tokens_translate)
    f5.write(str(score)+'\n')
f5.close()

24
24


In [12]:
from sklearn import preprocessing
import numpy as np

In [14]:
file = "/content/drive/MyDrive/MTdata/Hindi/yandex_metric/bleu/BLEU-yh.txt"
with open(file, 'r') as f:
  lines = [line.rstrip('\n') for line in f]
  print(lines)
  lst_int = [float(i) for i in lines]
  print(lst_int)
  m_score = list(lst_int)
  # normalized_arr = preprocessing.normalize([lst_int])
  # print(type(normalized_arr))
  # print(type(list(normalized_arr[0])))
  # print(type(normalized_arr[0]))
  # m_score = list(normalized_arr[0])
  # print(m_score)
  # up_score = []
  # for i in m_score:
  #   up_score.append(i)
  # print(up_score)
  

['8.286571670851008e-155', '8.57273414893938e-155', '0.5016513759455239', '0.24918622945587304', '3.5318487635731415e-78', '5.396466934950335e-155', '0.6407117598241614', '8.16437745974496e-155', '7.600793306264725e-78', '0.7172835948406505', '0.6528256693312096', '0.526054254065248', '0.3958275105780324', '0.1563968654529605', '0.5824157154892554', '0.6786502681586727', '0.53107253497887', '0.17181529671327242', '0.6930977286178778', '0.5339935148604845', '0.7140573910176907', '0.7544871870402462', '0.2304700957215317', '0.33932513407933634']
[8.286571670851008e-155, 8.57273414893938e-155, 0.5016513759455239, 0.24918622945587304, 3.5318487635731415e-78, 5.396466934950335e-155, 0.6407117598241614, 8.16437745974496e-155, 7.600793306264725e-78, 0.7172835948406505, 0.6528256693312096, 0.526054254065248, 0.3958275105780324, 0.1563968654529605, 0.5824157154892554, 0.6786502681586727, 0.53107253497887, 0.17181529671327242, 0.6930977286178778, 0.5339935148604845, 0.7140573910176907, 0.7544871

In [15]:
file = "/content/drive/MyDrive/MTdata/Hindi/yandex_score.txt"
with open(file, 'r') as f:
  lines = [line.rstrip('\n') for line in f]
  lst_dble = [float(i) for i in lines]
  print((lst_dble))

[0.8, 0.8, 0.9, 0.9, 0.6, 0.9, 0.7, 0.8, 0.9, 1.0, 0.9, 0.6, 0.8, 0.8, 0.9, 0.9, 0.9, 0.7, 0.8, 0.8, 0.8, 0.7, 0.8, 1.0]


In [16]:
import pandas as pd
from scipy.stats import pearsonr

corr, _ = pearsonr(m_score, lst_dble)
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.116
